## This code is the consolidation of just the relevant steps to train the model using Random Forest Algorithm and to save the corresponding pickle file locally.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import pickle

In [2]:
from sklearn.feature_selection import f_classif
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [3]:
ai_data=pd.read_csv('Downloads/XTern 2024 Artificial Intelegence Data Set - Xtern_TrainData.csv')

In [4]:
ai_data_df=pd.DataFrame(ai_data)

In [5]:
def Data_preparation(df):
    
    # Add Meal_Type column ={'Breakfast', 'Lunch', 'Dinner'}
    time_to_meal_type={}
    for i in list(df['Time'].unique()):
        if i in (8,9,10,11):
            time_to_meal_type[i]='Breakfast'
        elif i in (12,13,14,15):
            time_to_meal_type[i]='Lunch'
        else:
            time_to_meal_type[i]='Evening_Snack'
    Meal_type=[]
    for i in list(ai_data_df['Time']):
        Meal_type.append(time_to_meal_type[i])
    df['Meal_type']=Meal_type
    
    #One Hot encode all categorical variables except Order column
    
    
    one_hot_encoded_data = pd.get_dummies(ai_data_df, columns = ['Year', 'University','Major','Time', 'Meal_type'])
#     one_hot_encoded_data = one_hot_encoded_data.drop(columns=['Order'])
    
    return one_hot_encoded_data

In [6]:
one_hot_encoded_data=Data_preparation(ai_data_df)

In [7]:
X = one_hot_encoded_data.drop('Order', axis=1)
Y = one_hot_encoded_data['Order']
col_names= X.columns

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.15, stratify=Y)

In [9]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)
{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

In [10]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, Y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time=   0.8s
[CV] END bootstrap=False, max_depth=30, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=2000; total time=   0.0s
[CV] END bootstrap=False, max_depth=30, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=2000; total time=   0.0s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=1600; total time=   2.2s
[CV] END bootstrap=False, max_depth=100, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1000; total time=   1.8s
[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=2000; total time=   2.9s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimat

/Users/dhanyan/anaconda3/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1200; total time=   1.7s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=1600; total time=   2.2s
[CV] END bootstrap=False, max_depth=50, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1000; total time=   0.0s
[CV] END bootstrap=False, max_depth=50, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1000; total time=   0.0s
[CV] END bootstrap=False, max_depth=50, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1000; total time=   0.0s
[CV] END bootstrap=False, max_depth=10, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=1800; total time=   0.0s
[CV] END bootstrap=False, max_depth=10, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=1800; total time=   0.0s
[CV] END bootstrap=False, max_depth

/Users/dhanyan/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
123 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
55 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/dhanyan/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/dhanyan/anaconda3/lib/python3.11/site-packages/sklearn/base.py", line 1144, in wrapper
    estimator._validate_params()
  File "/Users/dhanyan/anaconda3/lib/python3.11/site-packages/sklearn/base.py", line 637, in _validate_params
    validate_parameter_constraints(
  Fil

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [11]:
y_pred=rf_random.predict(X_test)

In [12]:
#The model takes in these optimised parameters to produce the final output
rf_random.best_params_

{'n_estimators': 200,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 50,
 'bootstrap': True}

In [13]:
accuracy_score(Y_test, y_pred)

0.612

In [14]:
print(classification_report(Y_test, y_pred))


                                                          precision    recall  f1-score   support

                        Breaded Pork Tenderloin Sandwich       0.59      0.47      0.53        74
                                  Cornbread Hush Puppies       0.54      0.45      0.49        77
                                    Fried Catfish Basket       0.62      0.55      0.59        74
                        Hoosier BBQ Pulled Pork Sandwich       0.68      0.60      0.64        72
                 Indiana Buffalo Chicken Tacos (3 tacos)       0.50      0.45      0.47        74
    Indiana Corn on the Cob (brushed with garlic butter)       0.62      0.82      0.71        74
                                      Indiana Pork Chili       0.77      0.84      0.81        77
                                         Sugar Cream Pie       0.57      0.58      0.58        77
                                      Sweet Potato Fries       0.60      0.54      0.57        76
Ultimate Grilled Ch

In [15]:
#Create Pickle File
pickle.dump(rf_random,open('Downloads/Random_forest_model_Xtern_Dhanyan', 'wb'))

In [16]:
#Load the created pickle 
rf_model_loaded=pickle.load(open('Downloads/Random_forest_model_Xtern_Dhanyan', 'rb'))

In [17]:
#Feed the prepared test data into the loaded model
rf_model_loaded.predict(X_test)

array(['Indiana Pork Chili', 'Indiana Buffalo Chicken Tacos (3 tacos)',
       'Breaded Pork Tenderloin Sandwich', 'Sweet Potato Fries',
       'Hoosier BBQ Pulled Pork Sandwich',
       'Breaded Pork Tenderloin Sandwich', 'Sweet Potato Fries',
       'Sweet Potato Fries', 'Sugar Cream Pie',
       'Ultimate Grilled Cheese Sandwich (with bacon and tomato)',
       'Breaded Pork Tenderloin Sandwich',
       'Indiana Buffalo Chicken Tacos (3 tacos)',
       'Indiana Corn on the Cob (brushed with garlic butter)',
       'Sugar Cream Pie', 'Hoosier BBQ Pulled Pork Sandwich',
       'Sweet Potato Fries',
       'Indiana Corn on the Cob (brushed with garlic butter)',
       'Ultimate Grilled Cheese Sandwich (with bacon and tomato)',
       'Fried Catfish Basket', 'Indiana Pork Chili',
       'Cornbread Hush Puppies',
       'Indiana Buffalo Chicken Tacos (3 tacos)',
       'Breaded Pork Tenderloin Sandwich',
       'Hoosier BBQ Pulled Pork Sandwich', 'Sweet Potato Fries',
       'Indiana Cor